In [2]:
!pip install langchain_groq
!pip install chromadb

In [3]:
from langchain_groq import ChatGroq

llm= ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    groq_api_key='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'
)

response= llm.invoke("write a poem on lion")
print(response.content)

In savannah's golden light,
A king awakens, strong and bright,
The lion's mane, a fiery crown,
A symbol of power, in a regal town.

His eyes, like embers, burn with might,
As he surveys his kingdom, day and night,
His roar, a thunder, echoes far and wide,
A call to his pride, to gather by his side.

With muscles rippling, beneath his skin,
He stalks his prey, with stealthy kin,
A master hunter, with a patient heart,
He waits for the perfect moment to strike and part.

His mane, a shield, protects his neck,
As he battles foes, with a fierce attack,
His claws, like swords, slash and tear,
As he defends his pride, without a fear.

But when the sun sets, and the day is done,
The lion's heart, beats with a softer tone,
He curls up tight, with a gentle sigh,
And lets his pride, gather 'round to lie.

For though he's king, with a fierce might,
He's also a father, with a loving light,
And in his heart, a warmth does glow,
For the ones he loves, and the ones he'll know.


In [4]:
from langchain_community.document_loaders import WebBaseLoader 
loader = WebBaseLoader("https://jobs.nike.com/job/R-38844?from=job%20search%20funnel")
page_data=loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Software Engineer I

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer IBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does more than

In [10]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [11]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer I',
 'experience': '2+ years of professional experience in application engineering',
 'skills': ['JavaScript (Node.js, React.js, Vue.js, etc.)',
  'Agile processes',
  'Continuous integration',
  'Test automation',
  'AWS CloudFront',
  'AWS S3',
  'AWS Route 53',
  'AWS Certificate Manager',
  'AWS Lambda',
  'GitHub Co-pilot'],
 'description': 'The Digital Employee Experience Software Engineer I will be responsible for design, support, development, and testing of internal employee-facing websites and web applications.'}

In [7]:
type(json_res)

dict

In [12]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [17]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [56]:
links = collection.query(query_texts=['java developer'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [66]:
job = json_res
job['skills']

['JavaScript (Node.js, React.js, Vue.js, etc.)',
 'Agile processes',
 'Continuous integration',
 'Test automation',
 'AWS CloudFront',
 'AWS S3',
 'AWS Route 53',
 'AWS Certificate Manager',
 'AWS Lambda',
 'GitHub Co-pilot']

In [68]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Lakshay Pawar, a business development Manager at GROQ. GROQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of GROQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase GROQ's portfolio: {link_list}
        Remember you are Lakshay Pawar, BDM at GROQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Services for Your Digital Employee Experience

Dear Hiring Manager,

I came across the job description for a Software Engineer I role at your organization, and I'm excited to introduce GROQ, an AI & Software Consulting company that can help you fulfill your software engineering needs.

With our expertise in application engineering, we can support the design, development, and testing of internal employee-facing websites and web applications. Our team has extensive experience in JavaScript (Node.js, React.js, Vue.js, etc.), Agile processes, Continuous Integration, Test Automation, and AWS services (CloudFront, S3, Route 53, Certificate Manager, and Lambda).

At GROQ, we have a proven track record of delivering high-quality software solutions that meet our clients' specific needs. Our portfolio showcases our expertise in various technologies, including:

* DevOps: https://example.com/devops-portfolio
* Java: https://example.com/java-portfolio
* Kotlin 